In [ ]:
%matplotlib inline

# Overview

This describes how I processed the psicov dataset files to my liking: http://bioinfadmin.cs.ucl.ac.uk/downloads/contact_pred_datasets/

The `.tar` file that you download here should contain `aln` and `pdb` directories.

When we're done with our data cleaning, we're going to add an `aln_fasta` directory since the original alignment files are just in the psicov format. 

We'll also have a separate `aln_fasta_max1k` directory where no alignment will have more than 1001 sequences and the reference sequence will always be included in the set.

Offline we'll make some trees using FastTree2 which will reside in an eventual `raw_trees` directory.

Those trees will be rooted with the midpoint algorithm and put in `mp_root_trees` and then sent through IQTree and put in the `aln_fasta_max1k_iqtree` directory.

So you'll know you've done something right if at the end of all the data processing you have residing in some folder of interest:

1. `pdb/`
2. `aln/`
3. `aln_fasta/`
4. `aln_fasta_max1k/`
5. `raw_trees/`
6. `mp_root_trees/`
7. `aln_fasta_max1k_iqtree/`

My code assumes that these will all reside in `../Data/psicov_aln_pdb/`

**First the basic imports**

In [1]:
import glob
from Bio import SeqIO, Phylo

###I didn't use a random seed so your final results are going to be slightly different
###if you step through this entire pipeline from scratch. Alternatively you can just use the
###files I computed 
import random 

**Some globally relevant variable/s**

In [ ]:
pdb_directory = '../Data/psicov150_aln_pdb/pdb/' #Make sure that replacing "pdb/" with "aln/" exists!

# Clean `.fasta` files up a bit 
**(and truncate to max 1k seqs)**

In [ ]:
def replace_all(string, char_list_to_replace):
    """
    Just a basic and slow function to replace some weird characters should they appear
    """
    for char in char_list_to_replace:
        if char in string:
            string = string.replace(char, '-')
    return string

In [ ]:
############
invalid_characters = ['X', 'Z', 'B', 'U']

for pdb_file in glob.glob(pdb_directory+'*.pdb')[:]:
    print(pdb_file)
    pdb_seqio = list(SeqIO.parse(pdb_file, format='pdb-atom'))
    assert len(pdb_seqio) == 1
    #Grab the sequence!
    pdb_seq = str(pdb_seqio[0].seq)
    #Replace any of the above invalid characters
    pdb_seq = replace_all(pdb_seq, invalid_characters)
    #This file should exist from the original psicov directory
    aln_file = pdb_file.replace('/pdb/', '/aln/').replace('.pdb', '.aln')
    with open(aln_file, 'r') as infile:
        aln_seqs = infile.readlines()
        aln_seqs = [i.strip('\n') for i in aln_seqs]
        aln_seqs = [replace_all(i, invalid_characters) for i in aln_seqs]
    #We'll temporarily remove the reference sequence
    if pdb_seq in aln_seqs:
        aln_seqs.remove(pdb_seq)

    ####################################
    ###This shouldn't happen. And yet...
    if len(pdb_seq) != len(aln_seqs[0]) or len(pdb_seq) != len(aln_seqs[-1]):
        print('{} had an error'.format(pdb_file))
        continue
    ####################################

    
    
    ###Now we're writing the full alignments to fasta files with just numbers as their name
    with open(aln_file.replace('/aln/', '/aln_fasta/').replace('.aln', '.fasta'), 'w') as outfile:
        #First write the reference sequence
        outfile.write('>{}\n{}\n'.format('reference', pdb_seq))
        #Now everyone else
        for i, aln in enumerate(aln_seqs):
            outfile.write('>{}\n{}\n'.format(i, aln))
    
    #Sample the original sequences if there are too many of them
    if len(aln_seqs) > 1000.:
        aln_seqs = random.sample(aln_seqs, 1000)
    with open(aln_file.replace('/aln/', '/aln_fasta_max1k/').replace('.aln', '.fasta'), 'w') as outfile:
        #As before first write the reference
        outfile.write('>{}\n{}\n'.format('reference', pdb_seq))
        #And now the remaining sequences
        for i, aln in enumerate(aln_seqs):
            outfile.write('>{}\n{}\n'.format(i, aln))

# Offline: Run `FastTree`

**An important caveat here is that (I believe) that `FastTree` should be run in double precision mode. Which is to say, the normal `FastTree` probably is not the best for resolving such large alignments so make sure to run `FastTreeDbl`**

**You'll want to put these `.newick` tree results for each protein family into `raw_trees/`**

# Back online: Root `FastTree` results

The actual rooting algorithm *probably* isn't too big of a deal here so I'm sticking with the old standby of mid-point rooting. Though I'd note that I made a better/faster implementation than the stock BioPython version. And in the future it couldn't hurt to root these trees using a more complex algorithm.

The other important caveat here is that you need to put your FastTree newick output files into `raw_trees`. This will give them a quick root.

In [ ]:
import supporting_functions

In [ ]:
for fasttree_file in glob.glob('../Data/psicov150_aln_pdb/raw_trees/*.newick'):
    tree = Phylo.read(fasttree_file, 'newick')
    rooted_tree = supporting_functions.MP_root(tree)
    Phylo.write(tree, fasttree_file.replace('/raw_trees/', '/mp_root_trees/'),\
                'newick', format_branch_length='%1.16f')

# Offline: Run `IQTree`

**You'll want to put these results in `aln_fasta_max1k_iqtree/`**

**fin.**